# joining-tutorial.ipynb
# WESmith 11/5/22

In [ ]:
import pandas as pd
import numpy as  np
import os

### AN EXPLANATION OF MERGE() VS JOIN() AT:
### https://stackoverflow.com/questions/22676081/what-is-the-difference-between-join-and-merge-in-pandas

### "pandas.merge() is the underlying function used for all merge/join behavior."

## THE BELOW EXAMPLES FROM (BEST RESOURCE SO FAR):
### https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## CONCAT() SECTION TO BE ENTERED ...

## MERGE()

#### SINGLE UNIQUE KEYS

In [ ]:
left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

In [ ]:
right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [ ]:
left

In [ ]:
right

In [ ]:
result = pd.merge(left, right, on="key")
result

#### MULTIPLE KEYS THAT ARE DIFFERENT

In [ ]:
# WS multiple keys
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

In [ ]:
right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [ ]:
left

In [ ]:
right

#### INNER MERGE (DEFAULT)

In [ ]:
result = pd.merge(left, right, on=["key1", "key2"])
result

#### LEFT FRAME KEYS MERGE

In [ ]:
result = pd.merge(left, right, how="left", on=["key1", "key2"])
result

#### RIGHT FRAME KEYS MERGE

In [ ]:
result = pd.merge(left, right, how="right", on=["key1", "key2"])
result

#### OUTER MERGE

In [ ]:
result = pd.merge(left, right, how="outer", on=["key1", "key2"])
result

#### CROSS MERGE

In [ ]:
result = pd.merge(left, right, how="cross")
result

#### MULTI-INDEXED SERIES MERGE WITH DATAFRAME

In [ ]:
df = pd.DataFrame({"Let": ["A", "B", "C"], "Num": [1, 2, 3]})

In [ ]:
ser = pd.Series(
    ["a", "b", "c", "d", "e", "f"],
    index=pd.MultiIndex.from_arrays(
        [["A", "B", "C"] * 2, [1, 2, 3, 4, 5, 6]], names=["Let", "Num"]
    ),
)
ser

In [ ]:
df

In [ ]:
ser.reset_index()  # WS NICE WAY TO TURN A SERIES INTO A DATAFRAME

In [ ]:
df2 = pd.merge(df, ser.reset_index(), on=["Let", "Num"])
df2

# pick up here ...

## THE BELOW EXAMPLES FROM
### see https://www.datacamp.com/tutorial/joining-dataframes-pandas

In [ ]:
dummy_data1 = {
        'id': ['1', '2', '3', '4', '5'],
        'Feature1': ['A', 'C', 'E', 'G', 'I'],
        'Feature2': ['B', 'D', 'F', 'H', 'J']}
df1 = pd.DataFrame(dummy_data1, columns = ['id', 'Feature1', 'Feature2'])
df1

In [ ]:
dummy_data2 = {
        'id': ['1', '2', '6', '7', '8'],
        'Feature1': ['K', 'M', 'O', 'Q', 'S'],
        'Feature2': ['L', 'N', 'P', 'R', 'T']}
df2 = pd.DataFrame(dummy_data2, columns = ['id', 'Feature1', 'Feature2'])
df2

In [ ]:
dummy_data3 = {
        'id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'Feature3': [12, 13, 14, 15, 16, 17, 15, 12, 13, 23]}
df3 = pd.DataFrame(dummy_data3, columns = ['id', 'Feature3'])
df3

## CONCAT (NOTE: CONCAT() MAKES A FULL COPY)

In [ ]:
df_row = pd.concat([df1, df2])  # WS add data sets along rows: axis=0 by default; indexes retained
df_row

In [ ]:
df_row_reindex = pd.concat([df1, df2], ignore_index=True)  # WS reset indexes
df_row_reindex

In [ ]:
frames = [df1,df2]
df_keys = pd.concat(frames, keys=['x', 'y'])  # WS keep track of where data came from
df_keys

In [ ]:
df_keys.loc['y']  #  WS retrieving a particular dataset

In [ ]:
pieces = {'x': df1, 'y': df2}  # WS label datasets in a dictionary: get the same result
df_piece = pd.concat(pieces)
df_piece

In [ ]:
df_col = pd.concat([df1,df2], axis=1)  # WS concat along columns
df_col

## MERGE

In [ ]:
df_row

In [ ]:
df3

In [ ]:
df_merge_col = pd.merge(df_row, df3, on='id')
df_merge_col
# WS if an id is not in both datasets, it is missing in merge(): ie, 6,9,10,11 missing in result
# also, IDs 1,2 are each duplicated since they were duplicated in first dataset

In [ ]:
# WS if COLUMNS on which to merge have different NAMES (unlike this case): use:
df_merge_difkey = pd.merge(df_row, df3, left_on='id', right_on='id')
df_merge_difkey

## APPEND (DEPRECATED: USE CONCAT)

In [ ]:
add_row = pd.Series(['10', 'X1', 'X2', 'X3'],
                    index=['id','Feature1', 'Feature2', 'Feature3'])
add_row

In [ ]:
# WS append is deprecated, use concat
df_add_row = df_merge_col.append(add_row, ignore_index=True)
df_add_row

In [ ]:
# WS here is the new workaround to DataFrame.append() (append deprecated because it is very slow)
# see https://stackoverflow.com/questions/70837397/
#             good-alternative-to-pandas-append-method-now-that-it-is-being-deprecated
add_row_as_df = pd.DataFrame(add_row).T  # the key is to make a DF of the Series, and transpose it
add_row_as_df

In [ ]:
# WS then can concat it
df_add_row = pd.concat([df_merge_col, add_row_as_df], ignore_index=True)
df_add_row

## JOIN

### FULL OUTER JOIN USING MERGE()

In [ ]:
df1

In [ ]:
df2

In [ ]:
# WS this automatically creates new column labels showing the source of data;
# all records from both DataFrames and fill in NaNs where data is missing
df_outer = pd.merge(df1, df2, on='id', how='outer')
df_outer

In [ ]:
# WS can rename the suffixes for the new column labels
df_suffix = pd.merge(df1, df2, left_on='id',right_on='id',how='outer',suffixes=('_left','_right'))
df_suffix

### INNER JOIN USING MERGE()

In [ ]:
df_inner = pd.merge(df1, df2, on='id', how='inner')
df_inner

### RIGHT JOIN USING MERGE()

In [ ]:
df_right = pd.merge(df1, df2, on='id', how='right')
df_right

### LEFT JOIN USING MERGE()

In [ ]:
df_left = pd.merge(df1, df2, on='id', how='left')
df_left

### JOINING ON INDEX USING MERGE()

In [ ]:
df1

In [ ]:
df2

In [ ]:
# WS this joins on the indexes (ie, here 0,1,2,3,4)
df_index = pd.merge(df1, df2, right_index=True, left_index=True)
df_index

### TIME-SERIES MERGING USING MERGE_ASOF()

In [ ]:
trades = pd.DataFrame({
    'time': pd.to_datetime(['20160525 13:30:00.023',
                            '20160525 13:30:00.038',
                            '20160525 13:30:00.048',
                            '20160525 13:30:00.048',
                            '20160525 13:30:00.048']),
    'ticker': ['MSFT', 'MSFT','GOOG', 'GOOG', 'AAPL'],
    'price': [51.95, 51.95,720.77, 720.92, 98.00],
    'quantity': [75, 155,100, 100, 100]},
    columns=['time', 'ticker', 'price', 'quantity'])
trades

In [ ]:
quotes = pd.DataFrame({
    'time': pd.to_datetime(['20160525 13:30:00.023',
                            '20160525 13:30:00.023',
                            '20160525 13:30:00.030',
                            '20160525 13:30:00.041',
                            '20160525 13:30:00.048',
                            '20160525 13:30:00.049',
                            '20160525 13:30:00.072',
                            '20160525 13:30:00.075']),
    'ticker': ['GOOG', 'MSFT', 'MSFT','MSFT', 'GOOG', 'AAPL', 'GOOG','MSFT'],
    'bid': [720.50, 51.95, 51.97, 51.99,720.50, 97.99, 720.50, 52.01],
    'ask': [720.93, 51.96, 51.98, 52.00,720.93, 98.01, 720.88, 52.03]},
    columns=['time', 'ticker', 'bid', 'ask'])
quotes

In [ ]:
df_merge_asof = pd.merge_asof(trades, quotes,
              on='time',
              by='ticker')
df_merge_asof

In [ ]:
# WS specify a time tolerance: trade must be no more than 2ms after bid/ask quote
# one MSFT thrown out because trade is 8ms after bid/ask
df_merge_asof_tolerance = pd.merge_asof(trades, quotes,
              on='time',
              by='ticker',
              tolerance=pd.Timedelta('2ms'))
df_merge_asof_tolerance

## ANOTHER SOURCE: JOIN() VS MERGE()
### from https://sparkbyexamples.com/pandas/pandas-join-vs-merge-explained-differences/

In [ ]:
technologies = {
    'Courses' :["Spark","PySpark","Python","pandas"],
    'Fee'     :[20000,25000,22000,30000],
    'Duration':['30days','40days','35days','50days'],
              }
index_labels=['r1','r2','r3','r4']
df1 = pd.DataFrame(technologies,index=index_labels)
df1

In [ ]:
technologies2 = {
    'Courses' :["Spark","Java","Python","Go"],
    'Discount':[2000,2300,1200,2000]
              }
index_labels2=['r1','r6','r3','r5']
df2 = pd.DataFrame(technologies2,index=index_labels2)
df2

In [ ]:
# pandas left join two DataFrames by Index
# WS a suffix is REQUIRED, NO DEFAULTS
df3 = df1.join(df2, how='left', lsuffix="_left", rsuffix="_right")
df3

In [ ]:
# Merge two DataFrames by index using pandas.merge()
# WS same as join above, but suffix HAS DEFAULTS
df3 = pd.merge(df1, df2, left_index=True, right_index=True, how='left')
df3

In [ ]:
# pandas merge - inner join by Column
# WS NOTE how the index column is lost from the original DataFrames
df3=pd.merge(df1,df2)
df3

In [ ]:
# Merge DataFrames by Column
# WS same result as above, inner join
df3=pd.merge(df1,df2, on='Courses')
df3

In [ ]:
# WS added
df4 = df2.rename(columns={'Courses': 'My_Courses'})
df4

In [ ]:
# When column names are different
df3=pd.merge(df1,df4, left_on='Courses', right_on='My_Courses')
df3

In [ ]:
# WS added: delete redundant column
df5 = df3.drop('My_Courses', axis=1)
df5

In [ ]:
# Use join on column
df3 = df1.set_index('Courses').join(df2.set_index('Courses'), how='inner')
df3